In [1]:
import pandas as pd
import sys
import os

sys.path.append(os.path.abspath(os.path.join("../scripts")))

import load_data
from clean_data import dataCleaning

import cohere

# Entities extraction

In [2]:
extract_train = pd.read_json('../data/relations_dev.txt')
extract_test = pd.read_json('../data/relations_test.txt')

In [3]:
extract_train.head()

,document,tokens,relations
0,Bachelor's degree in Mechanical Engineering or...,"[{'text': 'Bachelor', 'start': 0, 'end': 8, 't...","[{'child': 4, 'head': 0, 'relationLabel': 'DEG..."
1,10+ years of software engineering work experie...,"[{'text': '10+ years', 'start': 0, 'end': 9, '...","[{'child': 4, 'head': 0, 'relationLabel': 'EXP..."
2,3+ years Swift & Objective-C and experience wi...,"[{'text': '3+ years', 'start': 0, 'end': 8, 't...","[{'child': 3, 'head': 0, 'relationLabel': 'EXP..."
3,8+ years experience in software engineering le...,"[{'text': '8+ years', 'start': 0, 'end': 8, 't...","[{'child': 5, 'head': 0, 'relationLabel': 'EXP..."
4,BS degree in Computer Science or related field...,"[{'text': 'BS', 'start': 0, 'end': 2, 'token_s...","[{'child': 3, 'head': 0, 'relationLabel': 'DEG..."


In [4]:
tokens_processed = []
for tokl in extract_train['tokens']:
    new = ""
    for d in tokl:
        new += f"{d['entityLabel']}: {d['text']} \n"
    
    tokens_processed.append([new])

In [5]:
extract_train['tokens_processed'] = tokens_processed

In [6]:
for i in range(len(extract_train)):
    extract_train['tokens_processed'][i] = extract_train['tokens_processed'][i][0]

In [7]:
prompt_extraction = ""
for i in range(len(extract_train)):
    prompt_extraction += f"Document: {extract_train.iloc[i][0]}\n{extract_train.iloc[i][-1]}" + "--" + "\n"

## Field extraction in the test set

In [8]:
for i in range(len(extract_test)):
    extract_test['document'][i] = extract_test['document'][i].replace('\n', '')

In [9]:
extract_test.head()

,document,tokens,relations
0,"Currently holding a faculty, industry, or gove...","[{'text': 'Ph.D.', 'start': 75, 'end': 80, 'to...","[{'child': 18, 'head': 14, 'relationLabel': 'D..."
1,2+ years experience in the online advertising ...,"[{'text': '2+ years', 'start': 1, 'end': 9, 't...","[{'child': 7, 'head': 1, 'relationLabel': 'EXP..."
2,BA/BS5+ years of program or project management...,"[{'text': '5+ years', 'start': 7, 'end': 15, '...","[{'child': 11, 'head': 5, 'relationLabel': 'EX..."
3,Currently enrolled in a full-time degree progr...,"[{'text': 'Ph.D.', 'start': 801, 'end': 806, '...","[{'child': 140, 'head': 137, 'relationLabel': ..."
4,Currently enrolled in a full-time degree progr...,"[{'text': 'Ph.D.', 'start': 801, 'end': 806, '...","[{'child': 140, 'head': 137, 'relationLabel': ..."


In [10]:
# for i in range(len(extract_test)):
#     prompt_test = prompt_extraction + f"Document: {extract_test.iloc[i][0]}\n"
#     print(f"Predicition number {i+1}")
#     co = cohere.Client('zXuOLIqQurZVDoERKbVcXjxlVJ41oFBW5Gdky02p')
#     response = co.generate(
#         model='xlarge',
#         prompt= prompt_test,
#         max_tokens=50,
#         temperature=0.9,
#         k=0,
#         p=0.75,
#         frequency_penalty=0,
#         presence_penalty=0,
#         stop_sequences=["--"],
#         return_likelihoods='NONE')
#     print('Prediction: {}'.format(response.generations[0].text))    
#     print('-----------------------------------------------')


In [11]:
# prompt_test = prompt_extraction + f"Document: {extract_test.iloc[0][0]}\n"
# print(prompt_test)

# Score prediction

In [12]:
data = load_data.load_excel_file('data/Example_data.xlsx', './', 'Sv1')

In [13]:
data = dataCleaning(data).clean_text('Title')
data = dataCleaning(data).clean_text('Description')
data = dataCleaning(data).clean_text('Body')

In [14]:
## Remove unnecessary columns
data_use = data[['Domain', 'Title', "Description", 'Body', 'Analyst_Average_Score']]
data_use.head()

,Domain,Title,Description,Body,Analyst_Average_Score
0,rassegnastampa.news,boris johnson using a taxpayerfunded jet for a...,…often trigger a protest vote that can upset…t...,boris johnson using a taxpayerfunded jet for a...,0.0
1,twitter.com,stumbled across an interesting case a woman fa...,stumbled across an interesting case a woman fa...,stumbled across an interesting case a woman fa...,0.0
2,atpe-tchad.info,marché résines dans les peintures et revêtemen...,…covid19…covid…covid…covid19 et post covid…cov...,le rapport d’étude de marché résines dans les ...,0.0
3,badbluetech.bitnamiapp.com,ai drives data analytics surge study finds,…hate raiders linked to automated harassment c...,how to drive the funnel through content market...,0.0
4,kryptogazette.com,triacetin vertrieb markt 2021 globale unterneh...,…abschnitten und endanwendungen organisatione...,global triacetin vertriebmarkt 2021 von herste...,0.0


In [15]:
# X = data_use.drop(['Analyst_Average_Score'], axis=1)
# y = data_use['Analyst_Average_Score']

# from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [16]:
# prompt_train = ""

# for j in range(len(X_train)):
#     for i in range(len(X_train.iloc[j].index)):
#         prompt_train = prompt_train + f"{X_train.iloc[j].index[i]}: {X_train.iloc[j][i]}" + "\n"
#     prompt_train = prompt_train + f"{'Analyst_Average_Score'}: {list(y_train)[j]}" + "\n"

#     prompt_train = prompt_train + "--" + "\n"

In [17]:
# for i in range(len(X_test.iloc[0].index)):
#     prompt_pred = prompt_train + f"{X_test.iloc[0].index[i]}: {X_test.iloc[0][i]}" + "\n"

# prompt_pred += "Analyst_Average_Score:"


In [18]:
# prompt_train = ""

# for j in range(len(data_use)):
#     for i in range(len(data_use.iloc[j].index)):
#         prompt_train = prompt_train + f"{data_use.iloc[j].index[i]}: {data_use.iloc[j][i]}" + "\n"

#     prompt_train = prompt_train + "--" + "\n"
